# Footprint the extents of raster input and output for SR-Lite

In [1]:
import rasterio
import rasterio.features
import rasterio.warp
from rasterio.plot import show
from rasterio import Affine, MemoryFile
from rasterio.enums import Resampling

import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import box
import fiona
from fiona.crs import from_epsg
import pprint

import glob
import os

import sys
sys.path.append('/efs/pmontesa/code/geoscitools')
sys.path.append('/home/pmontesa/code/geoscitools')
import maplib
import footprintlib
from footprintlib import *

# https://gis.stackexchange.com/questions/375577/how-do-i-write-out-a-mixed-geometry-geodataframe-to-a-geopackage
# https://gis.stackexchange.com/questions/187877/how-to-polygonize-raster-to-shapely-polygons

### Set up dictionaries of parameters

## Build a dictionary of the raster datasets for footprinting

In [2]:
dict_list = [

    # EVHR TOA: /adapt/nobackup/projects/ilab/projects/srlite/input
    {'location': 'local',
     'data_dir': "/adapt/nobackup/projects/ilab/projects/srlite/input",
     'data_source': 'EVHR',
     'image_type': '**',
     'search_tail_string': '/*toa.tif',
     'search_link_string': '',
     'DO_DATAMASK': False,
     'GET_ONLY_DATASETMASK': True,
     'MANY_CRS': True,
     'out_footprint_dir': '/adapt/nobackup/projects/ilab/projects/srlite/input',
     'footprint_name_SPLIT_CHAR': '_',
     'footprint_name_SPLIT_POS': 0,
     'aws_credential_fn': None
    },
    # CCDC input: /adapt/nobackup/projects/ilab/data/srlite/ccdc/CCDC_ALL
    {'location': 'local',
     'data_dir': "/adapt/nobackup/projects/ilab/data/srlite/ccdc/CCDC_ALL",
     'data_source': 'CCDC',
     'image_type': '',
     'search_tail_string': '/*ccdc.tif',
     'search_link_string': '',
     'DO_DATAMASK': False,
     'GET_ONLY_DATASETMASK': True,
     'MANY_CRS': True,
     'out_footprint_dir': '/adapt/nobackup/projects/ilab/data/srlite/ccdc', #'/adapt/nobackup/people/pmontesa/userfs02/_share',
     'footprint_name_SPLIT_CHAR': '_',
     'footprint_name_SPLIT_POS': 0,
     'aws_credential_fn': None
    },
    # SR-Lite cloudmasks
    {'location': 'local',
     'data_dir': "/adapt/nobackup/projects/ilab/data/srlite/cloudmask",
     'data_source': 'Cloudmask',
     'image_type': '**',
     'search_tail_string': '/*toa.cloudmask.v1.2.tif',
     'search_link_string': '',
     'DO_DATAMASK': False,
     'GET_ONLY_DATASETMASK': True,
     'MANY_CRS': True,
     'out_footprint_dir': '/adapt/nobackup/projects/ilab/data/srlite/cloudmask',
     'footprint_name_SPLIT_CHAR': '_',
     'footprint_name_SPLIT_POS': 0,
     'aws_credential_fn': None
    },

    # SR-lite output
    {'location': 'local',
     'data_dir': "/adapt/nobackup/projects/ilab/data/srlite/products/srlite-0.9.9-06132022-reproject_ALL_only_once_to_toa",
     'data_source': 'srlite-0.9.9',
     'image_type': '**',
     'search_tail_string': '/*sr-02m.tif',
     'search_link_string': '',
     'DO_DATAMASK': False,
     'GET_ONLY_DATASETMASK': True,
     'MANY_CRS': True,
     'out_footprint_dir': '/adapt/nobackup/people/pmontesa/userfs02/projects/srlite/misc',
     'footprint_name_SPLIT_CHAR': '_',
     'footprint_name_SPLIT_POS': 0,
     'aws_credential_fn': None
    },
]

### Run the footprinting code over an datalist to return and geodataframe and a geopackage **updated** 

https://stackoverflow.com/questions/67812512/rasterio-does-not-exist-in-the-file-system-and-is-not-recognized-as-a-support

In [3]:
######################
# Select a dictionary
d = dict_list[0]
TEST=False
######################
d

{'location': 'local',
 'data_dir': '/adapt/nobackup/projects/ilab/projects/srlite/input',
 'data_source': 'EVHR',
 'image_type': '**',
 'search_tail_string': '/*toa.tif',
 'search_link_string': '',
 'DO_DATAMASK': False,
 'GET_ONLY_DATASETMASK': True,
 'MANY_CRS': True,
 'out_footprint_dir': '/adapt/nobackup/projects/ilab/projects/srlite/input',
 'footprint_name_SPLIT_CHAR': '_',
 'footprint_name_SPLIT_POS': 0,
 'aws_credential_fn': None}

In [4]:
%%time

footprint_gdf_list = list() 

if d['image_type'] == '**' or d['image_type'] == '':
    layer_name = 'all'
else:
    layer_name = d['image_type']
    
#filename = 'footprints_' + d['data_source']+'_'+d['image_type']+'.gpkg'
filename = 'footprints_' + d['data_source']+'.gpkg'

#SEARCH_STR_LIST = ["/", sensor, image_type, ".tif"]
SEARCH_STR_LIST = ["/", d['image_type'], d['search_tail_string']]
keystring = d['search_link_string'].join(SEARCH_STR_LIST)
print(d['data_dir'] + keystring)

# Raster list
if d['location'] == 'local':
    r_fn_list = glob.glob(d['data_dir'] + keystring, recursive=True)
else:
    print('on s3...')
    r_fn_list = s3.glob(d['data_dir'] + keystring)
    r_fn_list = ['s3://' + f for f in r_fn_list]

if TEST:
    r_fn_list = r_fn_list[0:2]
    print(r_fn_list)
    filename = 'TEST_footprints_' + d['data_source']+'_'+d['image_type']+'.gpkg'

outprint_fn = os.path.join(d['out_footprint_dir'], filename)

print(f'{len(r_fn_list)} total inputs from ' + d['data_source'] )   

if d['location'] == 'local':
    # Raster files are local
    # Get a list geodataframes of indiv raster footprint geometries (both data and mask)
    f_gdf_lst = [raster_footprint(r_fn, DO_DATAMASK=d['DO_DATAMASK'], GET_ONLY_DATASETMASK=d['GET_ONLY_DATASETMASK'], R_READ_MODE='r', MANY_CRS=d['MANY_CRS']) for r_fn in r_fn_list]
if d['location'] == 's3':
    # Raster files are on s3
    # Get the rio AWS session needed to access the s3 rasters in order to footprint
    aws_session = get_rio_aws_session_from_creds(d['aws_credential_fn'])
    with rio.Env(aws_session):
        # Get a list geodataframes of indiv raster footprint geometries (both data and mask)
        f_gdf_lst = [raster_footprint(r_fn, DO_DATAMASK=d['DO_DATAMASK'], GET_ONLY_DATASETMASK=d['GET_ONLY_DATASETMASK'], R_READ_MODE='r') for r_fn in r_fn_list]
        
if len(f_gdf_lst)==0:
    print('\nNothing was footprinted.\n')
else:
    # Build a footprint database geodataframe from all rasters
    footprint_gdf = build_footprint_db(f_gdf_lst, TO_GCS=False, WRITE_GPKG=True, OUT_F_NAME=outprint_fn, OUT_LYR_NAME=layer_name)
    footprint_gdf['footprint_name'] = footprint_gdf['file'].str.split(d['footprint_name_SPLIT_CHAR'], expand=True)[d['footprint_name_SPLIT_POS']]
    footprint_gdf_list.append(footprint_gdf)


/adapt/nobackup/projects/ilab/projects/srlite/input/**/*toa.tif
457 total inputs from EVHR
Building GDF from list...
Wrote out spatial footprints to /adapt/nobackup/projects/ilab/projects/srlite/input/footprints_EVHR.gpkg
CPU times: user 33.7 s, sys: 2.3 s, total: 36 s
Wall time: 1min


### Select the footprints gdf

In [5]:
footprint_gdf = footprint_gdf_list[0]
#print(footprint_gdf.path.to_list())
#print(footprint_gdf.to_crs(4326).iloc[1])
print(footprint_gdf.iloc[0].path)
print(footprint_gdf.iloc[0].file)
#footprint_gdf[footprint_gdf.file=='MCD12Q2.h12v02.peak.bias.tif']#.to_crs(4326).geometry.to_json()
#footprint_gdf.crs.to_json()
footprint_gdf.to_crs(4326)

/adapt/nobackup/projects/ilab/projects/srlite/input/RailroadValley
WV02_20170530_P1BS_1030010069698700-toa.tif


,geometry,footprint_name,path,file,area_km2,area_ha
0,"POLYGON ((-115.59073 38.42258, -115.58742 38.5...",WV02,/adapt/nobackup/projects/ilab/projects/srlite/...,WV02_20170530_P1BS_1030010069698700-toa.tif,319.064695,31906.4695
1,"POLYGON ((-115.58281 38.42050, -115.57933 38.5...",WV02,/adapt/nobackup/projects/ilab/projects/srlite/...,WV02_20180429_P1BS_103001007D967C00-toa.tif,361.210602,36121.0602
2,"POLYGON ((-115.62939 38.31535, -115.62063 38.7...",WV01,/adapt/nobackup/projects/ilab/projects/srlite/...,WV01_20200714_P1BS_102001009A7AD800-toa.tif,1148.146469,114814.6469
3,"POLYGON ((-115.72236 38.35757, -115.71701 38.6...",WV01,/adapt/nobackup/projects/ilab/projects/srlite/...,WV01_20110505_P1BS_10200100120F8400-toa.tif,666.914360,66691.4360
4,"POLYGON ((-115.57390 38.39920, -115.56984 38.6...",WV02,/adapt/nobackup/projects/ilab/projects/srlite/...,WV02_20170918_M1BS_1030010071B61100-toa.tif,446.548260,44654.8260
...,...,...,...,...,...,...
452,"POLYGON ((-106.27828 32.81651, -106.28068 32.9...",QB02,/adapt/nobackup/projects/ilab/projects/srlite/...,QB02_20021020_P1BS_1010010001594B00-toa.tif,318.112480,31811.2480
453,"POLYGON ((-106.28034 32.86805, -106.28292 33.0...",WV01,/adapt/nobackup/projects/ilab/projects/srlite/...,WV01_20111020_P1BS_10200100176B6B00-toa.tif,417.898432,41789.8432
454,"POLYGON ((-106.23245 32.78147, -106.23508 32.9...",QB02,/adapt/nobackup/projects/ilab/projects/srlite/...,QB02_20090720_M1BS_1010010009F6C100-toa.tif,401.405600,40140.5600
455,"POLYGON ((-106.19794 32.85480, -106.20031 33.0...",WV02,/adapt/nobackup/projects/ilab/projects/srlite/...,WV02_20130302_P1BS_103001001FAB4D00-toa.tif,338.821399,33882.1399


## Map the footprints

In [27]:
maplib.MAP_LAYER_FOLIUM(footprint_gdf, LAYER_COL_NAME='file', LAYER_NAME = 'EVHR toa', zoom_start=2, lon_start=0, fig_h=300)

In [28]:
maplib.MAP_LAYER_FOLIUM(gpd.read_file('/adapt/nobackup/people/pmontesa/userfs02/projects/srlite/misc/footprints_srlite-0.9.9.gpkg'),
                        LAYER_COL_NAME='file', foliumMap = None, zoom_start=2, lon_start=0, fig_h=300,
                        LAYER_NAME = 'SR-Lite',
                        LAYER_STYLE_DICT = {'fillColor': 'black', 'color': 'blue', 'weight' : 0.75, 'opacity': 1, 'fillOpacity': 0.5})

In [21]:
import folium
from folium import Map, TileLayer, GeoJson, LayerControl, Icon, Marker, features, Figure, CircleMarker, plugins
Map_Figure=Figure(width=1000,height=250)
foliumMap = maplib.MAP_LAYER_FOLIUM(footprint_gdf, LAYER_COL_NAME='file', LAYER_NAME = 'EVHR toa', zoom_start=2, lon_start=0)
Map_Figure.add_child(foliumMap)
LAYER_COL_NAME = 'file'
LAYER_STYLE_DICT = {'fillColor': 'black', 'color': 'blue', 'weight' : 0.75, 'opacity': 1, 'fillOpacity': 0.5}
GEOJSON_LAYER = GeoJson(
            gpd.read_file('/adapt/nobackup/people/pmontesa/userfs02/projects/srlite/misc/footprints_srlite-0.9.9.gpkg'),
            name='SRLite',
            style_function=lambda x: LAYER_STYLE_DICT,
            tooltip=features.GeoJsonTooltip(
                fields=[LAYER_COL_NAME],
                aliases=[f'{LAYER_COL_NAME}:'],
            )
        )
GEOJSON_LAYER.add_to(foliumMap)

LayerControl().add_to(foliumMap)

foliumMap